Apriori算法的基本流程如下：

    初始化：设置参数，包括最小支持度阈值和最小置信度阈值。初始化L1，即频繁1项集的集合。扫描整个数据集，计算每个项集的支持度，将支持度大于或等于最小支持度阈值的项集加入L1。
    迭代：从L1出发，生成候选的k项集（k大于1），记为Ck。扫描整个数据集，对Ck中的每个候选进行计数，并计算每个候选的支持度。将支持度大于或等于最小支持度阈值的候选加入Lk。
    剪枝：在生成Ck的过程中，如果存在某个候选的子集的支持度大于该候选本身，那么该候选可以被剪掉，因为它的非空子集的支持度更高。
    终止：当无法找到更多的Lk时，算法结束。
    生成关联规则：对于每个频繁项集Lk，如果存在一个子集A和一个超集B，使得A和B都是频繁的，且A和B的交集非空，那么存在一个关联规则“A->B”。计算该规则的置信度，如果置信度大于或等于最小置信度阈值，则输出该规则。

In [ ]:
import pandas as pd  
from apyori import apriori  
# 读取数据集  
data = pd.read_csv('dataset.csv', header=None)  
# 将数据集转换为Apriori算法所需的格式  
data = data.iloc[:, 1:].values  
transactions = list(data)  
# 执行Apriori算法  
min_support = 0.3  
min_confidence = 0.7  
results = apriori(transactions, min_support=min_support, min_confidence=min_confidence)  
# 输出关联规则  
for result in results:  
    print(result)

Apriori通过不断的构造候选集、筛选候选集挖掘出频繁项集，需要多次扫描原始数据，当原始数据较大时，磁盘I/O次数太多，效率比较低下。这毫无疑问会成为Apriori算法最大的缺点一频繁项集发现的速度太慢。 FP-growth算法其实是在Apriori算法基础上进行了优化得到的算法，FPGrowth算法则只需扫描原始数据两遍，通过FP-tree数据结构对原始数据进行压缩，效率较高。

FP-growth算法只需要对数据库进行了两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori算法快。在小规模数据集上,这不是什么问题，但是当处理大规模数据集时，就会产生很大的区别。

FP-growth算法主要分为两个步骤：基于数据集构建FP树和从FP树递归挖掘频繁项集。FP-tree构建通过两次数据扫描，将原始数据中的事务压缩到一个FP-tree树，该FP-tree类似于前缀树，相同前缀的路径可以共用，从而达到压缩数据的目的。

In [3]:
import pyfpgrowth

transactions = [[1, 2, 3],
                [2, 4, 5],
                [1, 2, 4],
                [1, 3, 5],
                [2, 3],
                [1, 3, 5],
                [1, 2, 3, 5],
                [1, 2, 3]]

patterns = pyfpgrowth.find_frequent_patterns(transactions, 3)  

rules = pyfpgrowth.generate_association_rules(patterns, 0.5)

print(patterns)
print('===============')
print(rules)

{(5,): 4, (3, 5): 3, (1, 5): 3, (1, 3, 5): 3, (1,): 6, (2,): 6, (1, 2): 4, (2, 3): 4, (1, 2, 3): 3, (1, 3): 5}
{(5,): ((1, 3), 0.75), (1,): ((3,), 0.8333333333333334), (1, 3): ((2,), 0.6), (1, 5): ((3,), 1.0), (3, 5): ((1,), 1.0), (2,): ((1, 3), 0.5), (1, 2): ((3,), 0.75), (2, 3): ((1,), 0.75)}


(5,): 4 表示商品5在4个交易中出现。
(3,5)(3,5): 3 表示商品3和5的组合在3个交易中出现。
(1,5)(1,5): 3 表示商品1和5的组合在3个交易中出现。
(1,3,5)(1,3,5): 3 表示商品1、3和5的组合在3个交易中出现。
(1,)(1,): 6 表示商品1在6个交易中出现。
(2,)(2,): 6 表示商品2在6个交易中出现。
(1,2)(1,2): 4 表示商品1和2的组合在4个交易中出现。
(2,3)(2,3): 4 表示商品2和3的组合在4个交易中出现。
(1,2,3)(1,2,3): 3 表示商品1、2和3的组合在3个交易中出现。
(1,3)(1,3): 5 表示商品1和3的组合在5个交易中出现。

(5,): ((1, 3), 0.75) 表示如果购买了商品5，则购买商品1和3的概率为0.75。
(1,)(1,): ((3,), 0.8333333333333334) 表示如果购买了商品1，则购买商品3的概率为约0.833。
(1,3)(1,3): ((2,), 0.6) 表示如果购买了商品1和3，则购买商品2的概率为0.6。
(1,5)(1,5): ((3,), 1.0) 表示如果购买了商品1和5，则购买商品3的概率为1.0（即100%）。
(3,5)(3,5): ((1,), 1.0) 表示如果购买了商品3和5，则购买商品1的概率为1.0（即100%）。
(2,)(2,): ((1, 3), 0.5) 表示如果购买了商品2，则购买商品1和3的概率为0.5。
(1,2)(1,2): ((3,), 0.75) 表示如果购买了商品1和2，则购买商品3的概率为0.75。
(2,3)(2,3): ((1,), 0.75) 表示如果购买了商品2和3，则购买商品1的概率为0.75。